# Proof of Trust: Preventing Data Leakage with Skyulf

One of the biggest risks in Machine Learning is **Data Leakage**: when information from the test set (or the future) accidentally "leaks" into the training process. This creates models that look perfect during training but fail in production.

Common sources of leakage:
1.  **Imputation:** Filling missing values in the Test set using the mean of the *entire* dataset (including Test).
2.  **Scaling:** Normalizing Test data using the min/max of the *entire* dataset.
3.  **Target Encoding:** Encoding categorical features using the target mean of the *entire* dataset.

## The Skyulf Guarantee
Skyulf prevents this by design using the **Calculator / Applier** pattern.
- **Calculator:** Learns statistics *only* from the Training data.
- **Applier:** Applies those learned statistics to Test data blindly.

This notebook proves this behavior using the **Titanic** dataset. We will:
1.  Load the dataset.
2.  Split it into Train/Test.
3.  Run a Skyulf Pipeline.
4.  **Mathematically verify** that the Test data was processed using *only* Training statistics.

In [1]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# Ensure local skyulf-core is importable
try:
    import skyulf  # noqa: F401
except ImportError:
    here = Path.cwd()
    candidates = [here, here / 'skyulf-core', here.parent, here.parent / 'skyulf-core']
    for c in candidates:
        if (c / 'skyulf' / '__init__.py').exists():
            sys.path.insert(0, str(c))
            break

from skyulf import SkyulfPipeline
from skyulf.data.dataset import SplitDataset

# Load Titanic Dataset
print("Loading Titanic dataset...")
titanic = fetch_openml("titanic", version=1, as_frame=True)
df = titanic.frame

# Select relevant columns for demonstration
# 'sex': Categorical (needs encoding)
# 'age': Numeric with missing values (needs imputation)
# 'fare': Numeric (needs scaling)
# 'survived': Target
cols = ['sex', 'age', 'fare', 'survived']
df = df[cols].copy()

# Convert target to int
df['survived'] = df['survived'].astype(int)

print(f"Dataset Shape: {df.shape}")
print(df.head())
print("\nMissing Values:\n", df.isnull().sum())

Loading Titanic dataset...
Dataset Shape: (1309, 4)
      sex      age      fare  survived
0  female  29.0000  211.3375         1
1    male   0.9167  151.5500         1
2  female   2.0000  151.5500         0
3    male  30.0000  151.5500         0
4  female  25.0000  151.5500         0

Missing Values:
 sex           0
age         263
fare          1
survived      0
dtype: int64


In [13]:
# 2. Split Data
# We split BEFORE any processing to simulate a real-world scenario.
X = df.drop(columns=['survived'])
y = df['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create Skyulf Dataset
dataset = SplitDataset(
    train=pd.concat([X_train, y_train], axis=1),
    test=pd.concat([X_test, y_test], axis=1)
)

print(f"Train Shape: {dataset.train.shape}")
print(f"Test Shape: {dataset.test.shape}")

Train Shape: (916, 4)
Test Shape: (393, 4)


In [14]:
# 3. Define Pipeline
# We intentionally use methods that are prone to leakage if done wrong.

config = {
    "preprocessing": [
        # Imputation: Should use Train Mean
        {
            "name": "impute_age",
            "transformer": "SimpleImputer",
            "params": {"strategy": "mean", "columns": ["age"]}
        },
        # Scaling: Should use Train Mean/Std
        {
            "name": "scale_fare",
            "transformer": "StandardScaler",
            "params": {"columns": ["fare"]}
        },
        # Target Encoding: Should use Train Target Mean
        # This is the most dangerous one! If it sees Test target, it's 100% leakage.
        {
            "name": "encode_sex",
            "transformer": "TargetEncoder",
            "params": {"columns": ["sex"], "target_column": "survived"}
        }
    ],
    "modeling": {
        "type": "random_forest_classifier",
        "params": {"n_estimators": 10, "random_state": 42}
    }
}

pipeline = SkyulfPipeline(config)

# Fit the pipeline
# This runs fit() on Train and transform() on Test
print("Running pipeline...")
metrics = pipeline.fit(dataset, target_column="survived")
print("Pipeline execution complete.")

Running pipeline...
Pipeline execution complete.


In [19]:
# 4. Verification 1: Imputation Leakage
# Did we fill missing 'age' in Test with the Train mean?

# Get the fitted imputer from the pipeline
imputer_step = pipeline.feature_engineer.fitted_steps[0]
assert imputer_step['name'] == 'impute_age'

# The fitted transformer is stored in 'artifact'
artifact = imputer_step['artifact']
# print(f"DEBUG: artifact keys: {artifact.keys()}")

# Skyulf SimpleImputer stores learned values in 'fill_values'
fill_values = artifact['fill_values']
print(f"DEBUG: fill_values: {fill_values}")

# Calculate Train Mean manually
train_age_mean = X_train['age'].mean()
print(f"Train Age Mean: {train_age_mean:.4f}")

# Check what the imputer learned
# fill_values is a dict mapping column name to value
learned_mean = fill_values['age']
print(f"Imputer Learned Mean: {learned_mean:.4f}")

# Verify they match
np.testing.assert_almost_equal(train_age_mean, learned_mean)
print("✅ Imputation Proof: The imputer learned the mean ONLY from the Training set.")

DEBUG: fill_values: {'age': 29.10230905349794}
Train Age Mean: 29.1023
Imputer Learned Mean: 29.1023
✅ Imputation Proof: The imputer learned the mean ONLY from the Training set.


In [21]:
# 5. Verification 2: Scaling Leakage
# Did we scale 'fare' using Train Mean/Std?

scaler_step = pipeline.feature_engineer.fitted_steps[1]
assert scaler_step['name'] == 'scale_fare'

artifact = scaler_step['artifact']
# print(f"DEBUG: artifact keys: {artifact.keys()}")

# Calculate Train Stats manually
train_fare_mean = X_train['fare'].mean()
train_fare_std = X_train['fare'].std(ddof=0) # Sklearn uses ddof=0 for std
print(f"Train Fare Mean: {train_fare_mean:.4f}, Std: {train_fare_std:.4f}")

# Check what the scaler learned
columns = artifact['columns']
fare_idx = columns.index('fare')

learned_mean = artifact['mean'][fare_idx]
learned_scale = artifact['scale'][fare_idx]
print(f"Scaler Learned Mean: {learned_mean:.4f}, Scale: {learned_scale:.4f}")

# Verify
np.testing.assert_almost_equal(train_fare_mean, learned_mean)
np.testing.assert_almost_equal(train_fare_std, learned_scale)
print("✅ Scaling Proof: The scaler learned stats ONLY from the Training set.")

Train Fare Mean: 33.7092, Std: 52.8118
Scaler Learned Mean: 33.7092, Scale: 52.8118
✅ Scaling Proof: The scaler learned stats ONLY from the Training set.


In [24]:
# 6. Verification 3: Target Encoding Leakage
# Did we encode 'sex' using the Target Mean of the Training set?

encoder_step = pipeline.feature_engineer.fitted_steps[2]
assert encoder_step['name'] == 'encode_sex'

artifact = encoder_step['artifact']
encoder = artifact['encoder_object']

# Calculate Train Target Mean for 'sex' manually
# Group by 'sex' and calculate mean of 'survived'
train_sex_means = pd.concat([X_train, y_train], axis=1).groupby('sex', observed=True)['survived'].mean()
print("Train Target Means:\n", train_sex_means)

# Check what the encoder learned
# Sklearn TargetEncoder stores encodings in encodings_
# It corresponds to categories_
categories = encoder.categories_[0]
encodings = encoder.encodings_[0]

print("\nEncoder Learned Means:")
for cat, enc in zip(categories, encodings):
    print(f"  {cat}: {enc:.6f}")

# Verify
# Note: Sklearn TargetEncoder uses smoothing (shrinkage), so it won't be EXACTLY the raw mean.
# But it should be close, and definitely NOT influenced by Test data.
# To prove no leakage, we can check that it's NOT equal to the Full Dataset mean.

full_sex_means = pd.concat([X, y], axis=1).groupby('sex', observed=True)['survived'].mean()
print("\nFull Dataset Means (Leakage!):\n", full_sex_means)

# Check 'male'
male_train_mean = train_sex_means['male']
male_full_mean = full_sex_means['male']
male_encoded = encodings[list(categories).index('male')]

print(f"\nComparison for 'male':")
print(f"  Train Mean: {male_train_mean:.6f}")
print(f"  Full Mean:  {male_full_mean:.6f}")
print(f"  Encoded:    {male_encoded:.6f}")

# Assert that Encoded is closer to Train Mean than Full Mean (or just different from Full Mean)
# Since smoothing pulls it towards global mean, it might be tricky.
# But we can assert it's NOT the Full Mean.
assert abs(male_encoded - male_full_mean) > 1e-4, "Leakage detected! Encoded value matches Full Mean."
print("✅ Target Encoding Proof: The encoder did NOT use the full dataset statistics.")

Train Target Means:
 sex
female    0.694444
male      0.179054
Name: survived, dtype: float64

Encoder Learned Means:
  female: 0.693502
  male: 0.179250

Full Dataset Means (Leakage!):
 sex
female    0.727468
male      0.190985
Name: survived, dtype: float64

Comparison for 'male':
  Train Mean: 0.179054
  Full Mean:  0.190985
  Encoded:    0.179250
✅ Target Encoding Proof: The encoder did NOT use the full dataset statistics.


## Conclusion

We have mathematically verified that:
1.  **Imputation** on Test data used the **Train Mean**.
2.  **Scaling** on Test data used the **Train Mean/Std**.
3.  **Target Encoding** on Test data used the **Train Target Mean**.

This proves that **Skyulf pipelines are leakage-free by design**. The strict separation of `fit()` (Calculator) and `transform()` (Applier) ensures that no information from the Test set (or future data) can influence the model training.